In [1]:
from datetime import datetime, timezone

import numpy as np
import pandas as pd

from utils import get_api, get_users_from_user_ids, make_users_df

In [ ]:
# Get my followers (not necessary for now)

In [136]:
# Get all followers ids from target big account
api = get_api()
screen_name = 'MrCluni'
follower_ids = iterate_over_cursors(function=api.get_follower_ids, screen_name=screen_name)
print(len(follower_ids))

19126


In [2]:
# Get a subset of followers ids from target big account
api = get_api()
screen_name = 'therealbuni'
follower_ids = api.get_follower_ids(screen_name=screen_name)
print(len(follower_ids))

5000


In [4]:
num_users = 201
followers = get_users_from_user_ids(follower_ids[:num_users])
followers_df = make_users_df(followers, screen_name)

In [5]:
# Get users I already checked
df_users_played = pd.read_csv('./data/users_played.csv')

In [6]:
# Pick users to check
num_users = 10
friends_followers_ratio_threshold = 4
days_since_last_status_threshold = 0
friends_threshold = 30

friends_followers_ratio_mask = followers_df['friends_followers_ratio'] > friends_followers_ratio_threshold
days_since_last_status_mask = followers_df['days_since_last_status'] <= days_since_last_status_threshold
friends_mask = followers_df['friends_count'] > friends_threshold
not_played_mask = ~followers_df['screen_name'].isin(df_users_played['screen_name'])

df_to_follow = followers_df[friends_followers_ratio_mask & days_since_last_status_mask & friends_mask & not_played_mask].sample(num_users)

In [7]:
# View users to play
df_to_follow

,screen_name,location,followers_count,friends_count,created_at,favourites_count,statuses_count,last_status,friends_followers_ratio,time_processed,days_since_last_status,source_screen_name
13,xxxmoltarxxx1,Argentina en san clemente,55,359,Fri Nov 26 00:08:26 +0000 2021,42,24,2021-11-27 23:00:30+00:00,6.527273,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
154,jhilen_,,2,81,Sat Feb 08 13:48:29 +0000 2020,61,29,2021-11-27 14:24:05+00:00,40.500000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
142,CarlitaGerez,,6,108,Mon Apr 10 18:59:01 +0000 2017,429,122,2021-11-27 21:42:51+00:00,18.000000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
39,LucaRossini,Buenos Aires - Argentina,170,771,Sun Oct 24 14:35:27 +0000 2010,175031,49804,2021-11-27 23:46:58+00:00,4.535294,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
20,xeneize78332392,"Trelew, Argentina",4,105,Sat Nov 27 18:14:03 +0000 2021,1,1,2021-11-27 22:53:16+00:00,26.250000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
117,Victori79964577,,3,379,Sun Jan 13 12:09:02 +0000 2019,274,171,2021-11-27 17:29:19+00:00,126.333333,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
176,AndreaO71116183,,4,261,Thu Oct 21 20:29:42 +0000 2021,206,81,2021-11-27 20:42:26+00:00,65.250000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
99,juligaleanop,,6,123,Tue Jul 20 16:07:23 +0000 2021,2287,1495,2021-11-27 23:21:54+00:00,20.500000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
68,MarceloBauza7,,1,126,Fri Nov 26 15:54:20 +0000 2021,5,3,2021-11-27 04:21:15+00:00,126.000000,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
195,oleipla,Antártida,42,211,Thu Oct 24 00:46:21 +0000 2019,22544,1140,2021-11-27 21:55:24+00:00,5.023810,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni


In [8]:
# Save users played
df_to_follow.to_csv('./data/users_played.csv', mode='a', index=False, header=False)

In [9]:
# View users played
df_users_played = pd.read_csv('./data/users_played.csv')
df_users_played.tail(10)

,screen_name,location,followers_count,friends_count,created_at,favourites_count,statuses_count,last_status,friends_followers_ratio,time_processed,days_since_last_status,source_screen_name
28,xxxmoltarxxx1,Argentina en san clemente,55,359,Fri Nov 26 00:08:26 +0000 2021,42,24,2021-11-27 23:00:30+00:00,6.527272727272727,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
29,jhilen_,NaN,2,81,Sat Feb 08 13:48:29 +0000 2020,61,29,2021-11-27 14:24:05+00:00,40.5,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
30,CarlitaGerez,NaN,6,108,Mon Apr 10 18:59:01 +0000 2017,429,122,2021-11-27 21:42:51+00:00,18.0,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
31,LucaRossini,Buenos Aires - Argentina,170,771,Sun Oct 24 14:35:27 +0000 2010,175031,49804,2021-11-27 23:46:58+00:00,4.535294117647059,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
32,xeneize78332392,"Trelew, Argentina",4,105,Sat Nov 27 18:14:03 +0000 2021,1,1,2021-11-27 22:53:16+00:00,26.25,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
33,Victori79964577,NaN,3,379,Sun Jan 13 12:09:02 +0000 2019,274,171,2021-11-27 17:29:19+00:00,126.33333333333333,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
34,AndreaO71116183,NaN,4,261,Thu Oct 21 20:29:42 +0000 2021,206,81,2021-11-27 20:42:26+00:00,65.25,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
35,juligaleanop,NaN,6,123,Tue Jul 20 16:07:23 +0000 2021,2287,1495,2021-11-27 23:21:54+00:00,20.5,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
36,MarceloBauza7,NaN,1,126,Fri Nov 26 15:54:20 +0000 2021,5,3,2021-11-27 04:21:15+00:00,126.0,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
37,oleipla,Antártida,42,211,Thu Oct 24 00:46:21 +0000 2019,22544,1140,2021-11-27 21:55:24+00:00,5.023809523809524,2021-11-27 23:47:50.985170+00:00,0.0,therealbuni
